In [ ]:
import pandas as pd
import matplotlib
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/total_tRNA_acid_deg_oligosdegcoplex4.xlsx')
df1 = pd.read_excel('/Users/xyuan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724.xlsx')
df2 = pd.read_excel('/Users/xyuan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0821.xlsx')
df1 = thermo_df(df1)
df2 = thermo_df(df2)

In [ ]:
df_noad = pd.read_excel('/Users/xyuan/Documents/BioPharmaFinder/ZhangLab/oligossample1.xlsx')
df_noad = thermo_df(df_noad)
df_noad.shape

In [ ]:
plot_zone(df1)

In [ ]:
df_pre = pd.read_excel('~/Downloads/phe2.xlsx', 'pre')
df_post = pd.read_excel('~/Downloads/phe2.xlsx', 'post')
df_pre = match_dfs(df_pre, df_noad)
df_post = match_dfs(df_post, df2)
plotly_zones(df_pre, df_post, y='Vol')

bcr = base_calling_random(df_pre, homology=True)
plt, fig = plot_basecalling(*bcr, y='Vol')

In [ ]:
df_pre['Ratio'] = df_pre.Vol/df_pre.Vol.sum()
df_pre

In [ ]:
df_post.sort_values('Vol', ascending=False).iloc[1].Mass

In [ ]:
plotly_zone(df2[df2.Mass>23000], y='Vol')
top_mass = df2[df2.Mass>23000].sort_values('Vol', ascending=False).iloc[0].Mass
top_mass - C, df2[df2.Mass>23900].sort_values('Mass').iloc[:5]

In [ ]:
df_std_phe_76_5p = pd.read_excel('/Users/xyuan/Studio/tools/phe_76_5p.xlsx')
df_std_phe_76_3p = pd.read_excel('/Users/xyuan/Studio/tools/phe_76_3p.xlsx')
df_std_phe_75_3p = pd.read_excel('/Users/xyuan/Studio/tools/phe_75_3p.xlsx')

In [ ]:
df5p_1 = match_dfs(df_std_phe_76_5p, df1)
df5p_2 = match_dfs(df_std_phe_76_5p, df2)
df5p_1 = df5p_1[(df5p_1.Mass>3000)&(df5p_1.Mass<24000)]
df5p_2 = df5p_2[(df5p_2.Mass>3000)&(df5p_2.Mass<24000)]
df5p_2 = df5p_2[df5p_2.RT>1]

In [ ]:

df5p_1['Ratio'] = df5p_1.Vol / df5p_1.Vol.sum()
df5p_2['Ratio'] = df5p_2.Vol / df5p_2.Vol.sum()

df5p_1 = match_dfs(df5p_2, df5p_1)
df5p_2 = match_dfs(df5p_1, df5p_2)

plotly_zones(df5p_1, df5p_2, names=['Sample0724', 'Sample0821'])
plotly_zones(df5p_1, df5p_2, y='Ratio', names=['Sample0724', 'Sample0821'])
df5p_1.iloc[-10:], df5p_2.iloc[-10:]
ratio1 = df5p_1.Ratio.mean()
ratio2 = df5p_2.Ratio.mean()
ratio1, ratio2, 2.0*(ratio1 - ratio2)/(ratio1 + ratio2)


In [ ]:
def process_single_ladder(df, mass_pairs, full_mass, orientation=5, df_res=pd.DataFrame(), idx=-1):
    seats = int(full_mass // 320)
    col_base = 'Base' if idx < 0 else f'Base{idx}'
    col_mass = 'Mass' if idx < 0 else f'Mass{idx}'
    if df_res.empty:
        df_res = pd.DataFrame()
        df_res['position'] = range(1, seats+1)
        df_res.set_index('position', inplace=True)
    df_res[col_base] = ''
    df_res[col_mass] = ''
    
    df = df.fillna(0)
    for idx, row in df.iterrows():
        if row.Mass <= 0:
            continue
        pos = int(row.Mass // 320)
        if orientation == 3:
            pos = seats + 1 - pos
#             pos -= 1
        mass = df_res.loc[pos, col_mass] 
        if not mass:
            mass = row.Mass
        else:
            mass = '{}, {}'.format(mass, row.Mass)
        df_res.loc[pos, col_mass] = mass
        
    for t in mass_pairs:
        df_pair = df[df.Mass.isin(t)]
        if df_pair.empty:
            continue
        
        idx = df_pair['Mass'].idxmax()
        if orientation == 3:
            idx = df_pair['Mass'].idxmin()
        pos = int(df_pair.loc[idx].Mass // 320)
        if orientation == 3:
            pos = seats + 1 - pos
            pos -= 1
        base = df_res.loc[pos, col_base] 
        if not base:
            base = t[2]
        else:
            base = '{}, {}'.format(base, t[2])
        df_res.loc[pos, col_base] = base
    
    return df_res

In [ ]:
# fullmass = df_post.sort_values('Vol', ascending=False).iloc[1].Mass
# print(f'fullmass {fullmass}')
# bcr = base_calling_random(df5p_1)
# df_res = process_single_ladder(*bcr, fullmass, idx=0)

# fullmass = df_post.sort_values('Vol', ascending=False).iloc[0].Mass
# bcr = base_calling_random(df5p_2)
# df_res = process_single_ladder(*bcr, fullmass, df_res=df_res, idx=1)
# df_res.tail(20)

fullmass1 = df_post.sort_values('Vol', ascending=False).iloc[1].Mass
fullmass2 = df_post.sort_values('Vol', ascending=False).iloc[0].Mass
ladders = [(df5p_1, fullmass1, 5), (df5p_2, fullmass2, 5)]

def ladder_complementation(ladders):
    df_res = pd.DataFrame()
    for idx, ladder in enumerate(ladders):
        df = ladder[0]
        fullmass = ladder[1]
        print(f'processing fullmass {fullmass}')
        bcr = base_calling_random(df)
        df_res = process_single_ladder(*bcr, fullmass, df_res=df_res, idx=idx)

    return df_res

In [ ]:
df5p_1.head(5), df5p_2.head(5)
df5p_1.to_excel('~/Downloads/phe_data_sample0724.xlsx')
df5p_2.to_excel('~/Downloads/phe_data_sample0821.xlsx')

In [ ]:
arr1 = np.array(df5p_1.Ratio)
arr2 = np.array(df5p_2.Ratio)
a = np.array([arr1, arr2])
np.std(arr1, axis=1)